# Make Animation from png images

In [1]:
#Standard Header used on the projects

#first the major packages used for math and graphing
import numpy as np
import matplotlib.pyplot as plt
from cycler import cycler
import platform
from joblib import Parallel, delayed

#Custome graph format style sheet
plt.style.use('Prospectus.mplstyle')

#If being run by a seperate file, use the seperate file's graph format and saving paramaeters
#otherwise set what is needed
if not 'Saving' in locals():
    Saving = True
if not 'Titles' in locals():
    Titles = True
if not 'Ledgends' in locals():
    Ledgends = True
if not 'FFormat' in locals():
    FFormat = '.eps'

In [2]:
import multiprocessing
from joblib import Parallel, delayed
num_jobs=30

In [ ]:
## Task specific imports
import os as os
import keras.utils as image
import glob
from PIL import Image, ImageDraw
import pickle

In [13]:
import math
 

## Choosing Platform
Working is beinging conducted on several computers, and author needs to be able to run code on all without rewriting..  This segment of determines which computer is being used, and sets the directories accordingly.

In [4]:
HostName = platform.node()

if HostName == "Server":
    Computer = "Desktop"   
elif HostName[-6:] == 'wm.edu':
    Computer = "SciClone"
elif HostName == "SchoolLaptop":
    Computer = "LinLap"
elif HostName == "WTC-TAB-512":
    Computer = "PortLap"
else:
    Computer = "WinLap"
    
if Computer ==  "SciClone":
    rootfolder = '/sciclone/scr10/dchendrickson01/EFIT/'
    
else:
    asdfasdf

## Load data

In [5]:
cases = 'DoubleTestLarger'
Cases = [cases]
Views = ['Mid', 'Vert','Head','zplane25','zplane75','TopSurface','Energy','LeftSurface','RightSurface','TopEnergy','HeadEnd','WebEnd','HeadStart','WebStart']
imFolder=rootfolder+cases+'/'
fileName = imFolder+'MovementsR2MM.p'
print(fileName)

/sciclone/scr10/dchendrickson01/EFIT/DoubleTestLarger/MovementsR2MM.p


In [6]:
file=open(fileName,'rb')

In [7]:
temp = pickle.load(file)
file.close()

In [8]:
[Movements, MovementsX, MovementsY, MovementsZ] = temp

In [ ]:
fileName = imFolder+'Parameters.p'
file=open(fileName,'rb')
Parameters=pickle.load(file)
[AirCut,RailShape,Flaw,length1,width1,height1,imFolder,runName,Ties,frequency,
 Signalfrequency,cycle,FFunction,WheelLoad,pRatio1,yModulus1,rho1,cl1,ct1,Tsteps
 gl1,gw1,gh1,xmax,ymax,zmax,
 gridStartHead,gridStartWeb,gridEndWeb,gridEndFoot,gridStartHeadWidth,gridEndHeadWidth] = Parameters

file.close()
                 

## Image making functions

In [9]:
def EnergyFig(t, xStart, xEnd, yStart, yEnd, zStart, zEnd, Folder, v, figH):

    fig = plt.figure(figsize=(6,figH), dpi=300)

    Image = Movements[xStart:xEnd, yStart:yEnd, zStart:zEnd,t].T

    x,y,z = np.shape(Image)
    if x ==1:
        I2 = np.squeeze(Image, axis=(0,))
    if y ==1:
        I2 = np.squeeze(Image, axis=(1,))
    if z ==1:
        I2 = np.squeeze(Image, axis=(2,))

    plt.contourf(I2, v, cmap=plt.cm.jet)
    plt.savefig(imFolder+Folder+'/Energy'+str(t).zfill(5)+'.png')
    plt.close(fig)


In [10]:
# %%
def AnimationBook(xStart, xEnd, yStart, yEnd, zStart, zEnd, Folder):

    if xStart - xEnd == 0.0:
        figH = 6 * (yEnd - yStart) / (zEnd-zStart)
        xEnd+=1
    elif yStart - yEnd == 0.0:
        figH = 6 * (zEnd - zStart) / (xEnd-xStart)
        yEnd+=1
    elif zStart - zEnd == 0.0:
        figH = 6 * (yEnd - yStart) / (xEnd-xStart)
        zEnd+=1
    else:
        figH = 0

    if figH==0:
        print("Error, no Dimmension is a plane",yStart-yEnd)
    else:
        EMin = np.min(Movements[xStart:xEnd, yStart:yEnd, zStart:zEnd,:])
        EMax = np.max(Movements[xStart:xEnd, yStart:yEnd, zStart:zEnd,:])
        v = np.linspace(EMin, EMax, 30, endpoint=True)[0:20]

        print('About to make frames for ',Folder)
        temp = Parallel(n_jobs=num_jobs)(delayed(EnergyFig)(t, xStart, xEnd, yStart, yEnd, zStart, zEnd, Folder,v,figH) for t in range(Tsteps))

In [20]:
def CaseImage(Case, View):
    imfolder = rootfolder + Case + '/'+View+'/'
    files = glob.glob(os.path.join(imfolder, '*.png'))
    files.sort()
    images = []

    for file in files:
        images.append(image.load_img(file))
    try:
        images[0].save(rootfolder + Case + '/Testing'+View+'.gif',
           save_all=True, append_images=images[1:], optimize=False, duration=40, loop=0)
    except:
        pass
    return 0

## Specs from simulation
Need to add a sepereate pickle in the simulation to save out the parameters as a dictionary

#Dimmesnsion of simulation space in meters
length1 = 4
width1 = 0.1524 # 0.1524
height1 = 0.1524

#Choose ferquency to be used for excitment
frequency = 49720.0 

#MATERIAL 1 ((steel))
pRatio1 = 0.29                                    #poission's ratio in 
yModulus1 = 200 * (10**9)                           #youngs modulus in pascals
rho1 = 7800                                        #density in kg/m^3


#CALCULATED PARAMETERS FROM INPUTS

mu1 = yModulus1/(2*(1+pRatio1))                    #second Lame Parameter
lmbda1 = 2 * mu1 * pRatio1 / (1 - 2 * pRatio1)     #first Lame Parameter
#Calculate speed of longitudinal and transverse waves in material 1
cl1 = np.sqrt((lmbda1 + 2* mu1)/rho1)
ct1 = np.sqrt(mu1/rho1)

#calculate wave lengths for material 1
omegaL1 = cl1 / frequency
omegaT1 = ct1 / frequency

gs = (min(omegaL1, omegaT1) /12)    #grid step, omegaL2,omegaT2

#number of grid points
gl1 = int(math.ceil(length1 / gs)) +1       #length 
gw1 = int(math.ceil(width1 / gs)) +1       #width
gh1 = int(math.ceil(height1 / gs)) +1       #height

## for latter rail section, define the dimmmensions in terms of grid
HeadThickness = 0.05
WebThickness = 0.035
FootThickness = 0.03
HeadWidth = 0.102

relHeadThick = HeadThickness / height1
relWeb = WebThickness / width1
relFoot = FootThickness / height1
relHeadWidth = HeadWidth / width1

relStartHeadThick = 1 - relHeadThick
relStartWeb = 0.5 - (relWeb / 2.0)
relEndWeb = 0.5 + (relWeb / 2.0)
relStartHeadWidth = 0.5 - (relHeadWidth / 2.0)
relEndHeadWidth = 0.5 + (relHeadWidth / 2.0)


gridStartHead = round((gh1-3) * relStartHeadThick) + 1
gridStartWeb = round((gw1-3) * relStartWeb)  + 1
gridEndWeb = round((gw1-3) * relEndWeb)  + 1
gridEndFoot = round((gh1-3) * relFoot)  + 1
gridStartHeadWidth = round((gw1-3) * relStartHeadWidth)  + 1
gridEndHeadWidth = round((gw1-3)  * relEndHeadWidth)  + 1

np.shape(Movements)

xmax = np.shape(Movements)[0]
ymax = np.shape(Movements)[1]
zmax = np.shape(Movements)[2]
Tsteps =  np.shape(Movements)[3]

## make all the frames of all the cases

In [17]:
AnimationBook(0,xmax,gridStartWeb+1,gridStartWeb+1,gridEndFoot,gridStartHead,"WebStart")
plt.close('all')

AnimationBook(0,xmax,gridEndWeb-1,gridEndWeb-1,gridEndFoot,gridStartHead,"WebEnd")
plt.close('all')

AnimationBook(0,xmax,gridEndHeadWidth-1,gridEndHeadWidth-1,gridStartHead,zmax,"HeadEnd")
plt.close('all')

AnimationBook(0,xmax,gridStartHeadWidth+1,gridStartHeadWidth+1,gridStartHead,zmax,"HeadStart")
plt.close('all')

AnimationBook(0,xmax,gridStartHeadWidth,gridEndHeadWidth,zmax-3,zmax-3,"TopSurface")
plt.close('all')

AnimationBook(0,xmax,gridStartHeadWidth,gridStartHeadWidth,gridStartHead,zmax,"LeftSurface")
plt.close('all')

AnimationBook(0,xmax,gridStartWeb,gridStartWeb,gridEndFoot,gridStartHead,"RightSurface")
plt.close('all')

AnimationBook(0,xmax,0,ymax,int((zmax - gridStartHead)/2)+gridStartHead,int((zmax - gridStartHead)/2)+gridStartHead,"Head")
plt.close('all')

AnimationBook(0,xmax,gridStartWeb,gridEndWeb,int(zmax/2),int(zmax/2),"Mid")
plt.close('all')

AnimationBook(0,xmax,int(ymax/2),int(ymax/2),0,zmax,"Vert")
plt.close('all')

AnimationBook(xmax-4,xmax-4,0,ymax,0,zmax,"Energy")
plt.close('all')


About to make frames for  WebStart
About to make frames for  WebEnd
About to make frames for  HeadEnd
About to make frames for  HeadStart
About to make frames for  TopSurface
About to make frames for  LeftSurface
About to make frames for  RightSurface
About to make frames for  Head
About to make frames for  Mid
About to make frames for  Vert
About to make frames for  Energy


## Animate

In [21]:
for Case in Cases:
    nothiing = Parallel(n_jobs=30)(delayed(CaseImage)(Case, View) for View in Views)
print(nothiing)

2023-12-19 09:50:29.341906: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-19 09:50:30.259397: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-19 09:50:30.267515: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable th

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
